In [1]:
import re
import sys
import string
import math
from textblob import Word
from textblob import TextBlob
from collections import defaultdict

In [2]:
import collections

In [36]:
userProp = ["username","text","tweetid"]
postings = defaultdict(dict)
visit = defaultdict(dict)
Querylist = []
Length = defaultdict(dict)
docN = 0
TOPK = 100

In [37]:
def get_postings():
    global postings
    global docN
    f = open(r'/home/liks/Documents/实验/IR/实验03/tweets.txt')
    lines = f.readlines()#readlines()
    i = 0
    for line in lines:
        i += 1
        line = tokenize_tweet(line)
        tweetid = line[0]
        line.pop(0)
        Length[tweetid]=len(line)
       # termlist 是一个（词项：词频）的 列表
        termlist = [(k,v) for k,v in collections.Counter(line).items()]
        for te in termlist:
            if te[0] in postings.keys():
                postings[te[0]].append((tweetid,te[1]))
            else:
                postings[te[0]] = [(tweetid,te[1])]
    docN = i
    # 这样出来的posting就是一个这样的东西（字典） posting[] :
    #                                         |
    #                                        |->term1:[(tweetid,fq),(tweetid,fq)...]
    #                                       |->term2:[(tweetid,fq),(tweetid,fq)...]
    #                                       ->term3:[(tweetid,fq)...]

In [38]:
def idf(term):
    global postings
    global docN
    if term in postings.keys():
        return math.log(docN/len(postings[term]))
    else:
        return 0

In [39]:
def tfwght_query(term):
    global Querylist
#     [(k,v) for k,v in collections.Counter(a).items()]
    return (1+math.log(collections.Counter(Querylist)[term]))

In [40]:
def Search_document(documentID):
#     锁定一个document，计算这个document与当前query的打分
#     这个函数里，在get一下query的tf 直接都在这里算出来 因为tf_raw在这里
    global Querylist
    global Length
    tf_raw = []
    Score = 0
    for i in range(len(Querylist)):
        tfx = checkin(Querylist[i],documentID)
        tf_raw.append(tfx)
    for t in tf_raw:
        if t == 0:
            tf_wght = 0
        else:
            tf_wght = (1+math.log(t))
    NORMALIZE = math.sqrt(sum(w**2 for w in tf_raw))
    term_normalized = [ num/NORMALIZE for num in tf_raw]
    for i in range(len(Querylist)):
        Score += tfwght_query(Querylist[i])*idf(Querylist[i])*term_normalized[i]
#     Score = Score/Length[documentID]
    return Score

In [41]:
def checkin(term,documentID):
#     计算document中是否包含term，如果包含 返回包含多少 没有就返回0
    global postings
    if term not in postings.keys():
        return 0
#     elif documentID in [te[1] for te in postings[term] ]
    else:
        for docid in postings[term]:
            if docid[0]==documentID:
                return docid[1]
    return 0        

In [42]:
class Document:
#     posting 是badge中保存每一个文档id所用到的类
    def __init__(self):
        self.documentid = 0
        self.tf = 0

In [43]:
class badge:
#     这是postings中的一个值
#     postings -> term:badge{df,postings{documentID,tf}} （df Query里要用到）
    def __init__(self):
        self.df = 0
        self.Document = None

In [44]:
def tokenize_tweet(string):
#     返回一个 列表 ，包括 username、tweetcontent和tweetid     最后的结果中不包含这三个字段
#     列表第一个值是tweetid 然后是content     [tweetid,content]
    string=string.lower()
    a = string.index("username")
    b = string.index("clusterno")
    c = string.rindex("tweetid")-1
    d = string.rindex("errorcode")
    e = string.index("text")
    f = string.index("timestr")-3
    #提取用户名、tweet内容和tweetid三部分主要信息
    string = string[c:d]+string[a:b]+string[e:f]
    # print(string)
    terms=TextBlob(string).words.singularize()
    # print(terms)

    result=[]
    for word in terms:
        expected_str = Word(word)
        expected_str = expected_str.lemmatize("v")
        if expected_str not in userProp:
            result.append(expected_str)
    return result

In [45]:
def token(query):
#     返回的就是Querylist
    query = query.lower()
    terms=TextBlob(query).words.singularize()

    result=[]
    for word in terms:#query和term做同样的处理
        expected_str = Word(word)
        expected_str = expected_str.lemmatize("v")
        result.append(expected_str)
    return result

In [46]:
def Search(query):
    global Querylist
    global visit
    global TOPK
    Querylist = token(query)
#     print(Querylist)
    for TERM in Querylist:
#         print('now it turns to %s'%TERM)
        if TERM in postings.keys():
#             print('%s in postings.keys()'%TERM)
            for doc in postings[TERM]:
                if doc[0] not in visit.keys():
                    visit[doc[0]] = Search_document(doc[0])

    return(sorted(visit.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)[:TOPK])
    

In [47]:
get_postings()
# 启动函数

In [48]:
Search('Manufacturers of industrial controllers')

[('28977078074343425', 10.828621111080203),
 ('625192875345768448', 8.02446940301423),
 ('31831410260058113', 8.02446940301423),
 ('317377292047376387', 8.02446940301423),
 ('30172846525251585', 8.02446940301423),
 ('30172275693068288', 8.02446940301423),
 ('30147216362446848', 8.02446940301423),
 ('29980635686772738', 8.02446940301423),
 ('34780760116305920', 7.619004294906064),
 ('34635341382025216', 7.619004294906064),
 ('33659477823590400', 7.619004294906064),
 ('33192271625064448', 7.619004294906064),
 ('33191722632613889', 7.619004294906064),
 ('33133626308698113', 7.619004294906064),
 ('32955696215498752', 7.619004294906064),
 ('32874160988356608', 7.619004294906064),
 ('32652240711852032', 7.619004294906064),
 ('30996863431745536', 7.619004294906064),
 ('302154195702800384', 7.619004294906064),
 ('303966009197469696', 4.915531143865067),
 ('626540723328970752', 1.6862287441467623),
 ('626514865423785984', 1.6862287441467623),
 ('626513707800117248', 1.6862287441467623),
 ('6265

In [23]:
print([postings])

AttributeError: 'list' object has no attribute 'keys'

In [49]:
def search_from_file(infile,outfile):
    infopen = open(infile,'r',encoding='utf-8')
    outopen = open(outfile,'w',encoding='utf-8')
    
    lines = infopen.readlines()
    for line in lines:
        line = line.strip().split("\t")
        ID = line[0]
        listtemp = [te[0] for te in Search(line[1])]
        for i in listtemp:
            outopen.write(str(ID) + ' ' + i + '\n')
    infopen.close()
    outopen.close()

In [50]:
search_from_file('/home/liks/Desktop/query.txt','/home/liks/Documents/实验/IR/实验03/MyAns.txt')